## *Importing all the required libraries and functions*

In [3]:
import pandas as pd
from pymongo import MongoClient
import pymongo
import time
import numpy as np
import scipy.linalg
import tensorflow as tf
from copy import deepcopy
from threading import Lock
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


## *Load data*

In [4]:
t = pd.read_csv('cardio_train.csv', sep=';')
t.head()
t['age'] = (t['age'] / 365).astype('int')
t.drop('id', axis=1, inplace=True)
x = t.drop('cardio', axis=1)
y = t['active']
np.unique(y, return_counts=True)
y = pd.get_dummies(y) # OHE
t.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,50,2,168,62.0,110,80,1,1,0,0,1,0
1,55,1,156,85.0,140,90,3,1,0,0,1,1
2,51,1,165,64.0,130,70,3,1,0,0,0,1
3,48,2,169,82.0,150,100,1,1,0,0,1,1
4,47,1,156,56.0,100,60,1,1,0,0,0,0


## *Input datas are converted into JSON format and stored into MongoDBNoSQL database.*

In [5]:

# Read the semicolon-delimited CSV dataset using pandas
dataset_path = 'cardio_train.csv'
data = pd.read_csv(dataset_path, delimiter=';')

# Convert data to JSON format
data_json = data.to_dict(orient='records')

# MongoDB connection
def connect_to_mongodb():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['heart_disease_database']
    collection = db['heart_failure_data']
    return collection

# Connect to MongoDB
collection = connect_to_mongodb()

# Insert data into MongoDB
collection.insert_many(data_json)
print("CSV converted to JSON.")
print(f"{len(data_json)} records inserted into MongoDB.")


CSV converted to JSON.
70000 records inserted into MongoDB.


## *Retrieve and display the datas from mongodb*

In [6]:
# MongoDB connection
def connect_to_mongodb():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['heart_disease_database']
    collection = db['heart_failure_data']
    return collection

# Connect to MongoDB
collection = connect_to_mongodb()

# Retrieve and display the first 5 records
limit = 5
for record in collection.find().limit(limit):
    print(record)

print(f"Displayed {limit} records.")

{'_id': ObjectId('64e32b4f60291ad11c2061c4'), 'id': 0, 'age': 18393, 'gender': 2, 'height': 168, 'weight': 62.0, 'ap_hi': 110, 'ap_lo': 80, 'cholesterol': 1, 'gluc': 1, 'smoke': 0, 'alco': 0, 'active': 1, 'cardio': 0}
{'_id': ObjectId('64e32b4f60291ad11c2061c5'), 'id': 1, 'age': 20228, 'gender': 1, 'height': 156, 'weight': 85.0, 'ap_hi': 140, 'ap_lo': 90, 'cholesterol': 3, 'gluc': 1, 'smoke': 0, 'alco': 0, 'active': 1, 'cardio': 1}
{'_id': ObjectId('64e32b4f60291ad11c2061c6'), 'id': 2, 'age': 18857, 'gender': 1, 'height': 165, 'weight': 64.0, 'ap_hi': 130, 'ap_lo': 70, 'cholesterol': 3, 'gluc': 1, 'smoke': 0, 'alco': 0, 'active': 0, 'cardio': 1}
{'_id': ObjectId('64e32b4f60291ad11c2061c7'), 'id': 3, 'age': 17623, 'gender': 2, 'height': 169, 'weight': 82.0, 'ap_hi': 150, 'ap_lo': 100, 'cholesterol': 1, 'gluc': 1, 'smoke': 0, 'alco': 0, 'active': 1, 'cardio': 1}
{'_id': ObjectId('64e32b4f60291ad11c2061c8'), 'id': 4, 'age': 17474, 'gender': 1, 'height': 156, 'weight': 56.0, 'ap_hi': 100, 

In [7]:
class UKF:
    def __init__(self, num_states, process_noise, initial_state, initial_covar, alpha, k, beta, iterate_function):
        """
        Initializes the unscented kalman filter
        :param num_states: int, the size of the state
        :param process_noise: the process noise covariance per unit time, should be num_states x num_states
        :param initial_state: initial values for the states, should be num_states x 1
        :param initial_covar: initial covariance matrix, should be num_states x num_states, typically large and diagonal
        :param alpha: UKF tuning parameter, determines spread of sigma points, typically a small positive value
        :param k: UKF tuning parameter, typically 0 or 3 - num_states
        :param beta: UKF tuning parameter, beta = 2 is ideal for gaussian distributions
        :param iterate_function: function that predicts the next state
                    takes in a num_states x 1 state and a float timestep
                    returns a num_states x 1 state
        """
        self.n_dim = int(num_states)
        self.n_sig = 1 + num_states * 2
        self.q = process_noise
        self.x = initial_state
        self.p = initial_covar
        self.beta = beta
        self.alpha = alpha
        self.k = k
        self.iterate = iterate_function

        self.lambd = pow(self.alpha, 2) * (self.n_dim + self.k) - self.n_dim

        self.covar_weights = np.zeros(self.n_sig)
        self.mean_weights = np.zeros(self.n_sig)

        self.covar_weights[0] = (self.lambd / (self.n_dim + self.lambd)) + (1 - pow(self.alpha, 2) + self.beta)
        self.mean_weights[0] = (self.lambd / (self.n_dim + self.lambd))

        for i in range(1, self.n_sig):
            self.covar_weights[i] = 1 / (2*(self.n_dim + self.lambd))
            self.mean_weights[i] = 1 / (2*(self.n_dim + self.lambd))

        self.sigmas = self.__get_sigmas()

        self.lock = Lock()

    def __get_sigmas(self):
        """generates sigma points"""
        ret = np.zeros((self.n_sig, self.n_dim))

        tmp_mat = (self.n_dim + self.lambd)*self.p

        # print spr_mat
        spr_mat = scipy.linalg.sqrtm(tmp_mat)

        ret[0] = self.x
        for i in range(self.n_dim):
            ret[i+1] = self.x + spr_mat[i]
            ret[i+1+self.n_dim] = self.x - spr_mat[i]

        return ret.T

    def update(self, states, data, r_matrix):
       
        self.lock.acquire()

        num_states = len(states)

        # create y, sigmas of just the states that are being updated
        sigmas_split = np.split(self.sigmas, self.n_dim)
        y = np.concatenate([sigmas_split[i] for i in states])

        # create y_mean, the mean of just the states that are being updated
        x_split = np.split(self.x, self.n_dim)
        y_mean = np.concatenate([x_split[i] for i in states])

        # differences in y from y mean
        y_diff = deepcopy(y)
        x_diff = deepcopy(self.sigmas)
        for i in range(self.n_sig):
            for j in range(num_states):
                y_diff[j][i] -= y_mean[j]
            for j in range(self.n_dim):
                x_diff[j][i] -= self.x[j]

        # covariance of measurement
        p_yy = np.zeros((num_states, num_states))
        for i, val in enumerate(np.array_split(y_diff, self.n_sig, 1)):
            p_yy += self.covar_weights[i] * val.dot(val.T)

        # add measurement noise
        p_yy += r_matrix

        # covariance of measurement with states
        p_xy = np.zeros((self.n_dim, num_states))
        for i, val in enumerate(zip(np.array_split(y_diff, self.n_sig, 1), np.array_split(x_diff, self.n_sig, 1))):
            p_xy += self.covar_weights[i] * val[1].dot(val[0].T)

        k = np.dot(p_xy, np.linalg.inv(p_yy))

        y_actual = data

        self.x += np.dot(k, (y_actual - y_mean))
        self.p -= np.dot(k, np.dot(p_yy, k.T))
        self.sigmas = self.__get_sigmas()

        self.lock.release()

    

In [8]:
# Connect to MongoDB
client = pymongo.MongoClient("localhost", 27017)
db = client["heart_disease_database"]
collection = db["heart_failure_data"]

# Retrieve data from MongoDB
documents = collection.find()
# Retrieve data from MongoDB
data_from_mongo = list(collection.find())

In [9]:
# Assume that each document has features and a label ('normal' or 'abnormal')
X = []  # Features
Y = []  # Labels

for document in data_from_mongo:
    features = [document["id"],
        document["age"],
        document["gender"],
        document["height"],
        document["weight"],
        document["ap_hi"],
        document["ap_lo"],document["cholesterol"],document["gluc"],document["smoke"],document["alco"],
        document["active"]]  # Add more features as needed
    label = 1 if document['cardio'] == 1 else 0
    
    X.append(features)
    Y.append(label)



# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## **Interference-Tolerant Fast Convergence Zeroing Neural Network (ITFCZNN)**

In [10]:
# Define the ITFCZNN architecture
def itfcznn_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
    # Input Layer
    tf.keras.layers.Dense(units=x.shape[1], activation='relu', input_shape=(11,)),
    # Hidden Layer
    tf.keras.layers.Dense(units=30, activation='relu'),
    tf.keras.layers.Dropout(rate=.2),
    tf.keras.layers.Dense(units=40, activation='relu'),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(units=50, activation='relu'),
    tf.keras.layers.Dropout(.4),
    tf.keras.layers.Dense(units=40, activation='relu'),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(units=30, activation='relu'),
    tf.keras.layers.Dropout(.2),
#     tf.keras.layers.Dense(units=40, activation='relu'),
    # Output Layer
    tf.keras.layers.Dense(units=2, activation='softmax')
    ])
    return model

# Define input shape and number of classes
input_shape = (11,)
num_classes=2  
# Create the model
model = itfcznn_model(input_shape, num_classes)
    # Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
                       loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                       metrics=tf.keras.metrics.CategoricalAccuracy())

# Train the model
model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=50)
    

   

Epoch 1/15
1050/1050 [==============================] - 14s 4ms/step - loss: 0.5630 - categorical_accuracy: 0.7959 - val_loss: 0.4961 - val_categorical_accuracy: 0.8039
Epoch 2/15
1050/1050 [==============================] - 4s 4ms/step - loss: 0.4983 - categorical_accuracy: 0.8031 - val_loss: 0.4863 - val_categorical_accuracy: 0.8039
Epoch 3/15
1050/1050 [==============================] - 4s 3ms/step - loss: 0.2606 - categorical_accuracy: 0.8875 - val_loss: 0.0293 - val_categorical_accuracy: 0.9887
Epoch 4/15
1050/1050 [==============================] - 5s 4ms/step - loss: 0.0283 - categorical_accuracy: 0.9928 - val_loss: 0.0137 - val_categorical_accuracy: 0.9953
Epoch 5/15
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0197 - categorical_accuracy: 0.9940 - val_loss: 0.0118 - val_categorical_accuracy: 0.9961
Epoch 6/15
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0181 - categorical_accuracy: 0.9956 - val_loss: 0.0135 - val_categorical_accuracy

## *Prediction*

In [11]:
# Generate predictions
X_train, x_test, y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
itfcznn_model = tf.keras.Sequential([
           tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
           tf.keras.layers.Dense(32, activation='relu'),
         tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
        ])
predictions = itfcznn_model.predict(X_test_scaled)

# Apply threshold for binary classification
threshold = 0.5
binary_predictions = (predictions < threshold).astype(int)

# Map binary predictions to labels
class_mapping = {1: 'presence', 0: 'absence'}
predicted_labels = [class_mapping[int(prediction)] for prediction in binary_predictions]

# Assuming each prediction corresponds to a document with an ID
document_ids = x_test[:, 0]  # Assuming 'id' is the first feature in your X array

# Create a list of dictionaries with ID and prediction result
results = [{'id': int(document_id), 'prediction': label} for document_id, label in zip(document_ids, predicted_labels)]
results

547/547 [==============================] - 1s 1ms/step


[{'id': 66728, 'prediction': 'presence'},
 {'id': 69098, 'prediction': 'presence'},
 {'id': 59185, 'prediction': 'presence'},
 {'id': 49288, 'prediction': 'presence'},
 {'id': 62481, 'prediction': 'presence'},
 {'id': 16605, 'prediction': 'presence'},
 {'id': 35209, 'prediction': 'presence'},
 {'id': 32015, 'prediction': 'absence'},
 {'id': 1307, 'prediction': 'absence'},
 {'id': 80437, 'prediction': 'presence'},
 {'id': 37048, 'prediction': 'presence'},
 {'id': 45001, 'prediction': 'presence'},
 {'id': 6624, 'prediction': 'presence'},
 {'id': 24101, 'prediction': 'absence'},
 {'id': 5972, 'prediction': 'presence'},
 {'id': 93793, 'prediction': 'absence'},
 {'id': 5352, 'prediction': 'presence'},
 {'id': 44803, 'prediction': 'presence'},
 {'id': 90475, 'prediction': 'presence'},
 {'id': 14122, 'prediction': 'presence'},
 {'id': 58765, 'prediction': 'presence'},
 {'id': 4197, 'prediction': 'absence'},
 {'id': 92516, 'prediction': 'presence'},
 {'id': 94854, 'prediction': 'absence'},
 {'

In [ ]:
try: 
  user_choice = int(input("Enter the id"))
except Exception:
  print('Invalid input:Please enter the valid input')
  user_choice = int(input("Enter the id"))
def predict_result(user_choice):
  for result in results:
    if (result['id'] == user_choice):
      return result
print("ID not found")
result = predict_result(user_choice)
result

## **Prediction stored in MongoDBNoSQL database.**

In [12]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB connection string
db = client["heart_disease_database"]  # Replace with your database name
collection = db["prediction_results"]  # Replace with your desired collection name

# Insert results into the MongoDB collection
collection.insert_one(result)

# Close the MongoDB connection
client.close()
print("Predictions saved in MongoDB.")

Predictions saved in MongoDB.


## *Gannet Optimization Algorithm (GOA)*

In [13]:

# Problem-specific parameters
num_variables = 10  # Number of optimization variables
lower_bound = -10  # Lower bound of variables
upper_bound = 10   # Upper bound of variables

# Algorithm parameters
num_gannets = 50    # Number of gannets (population size)
max_iterations = 100  # Maximum number of iterations
a_max = 2            # Maximum value for acceleration coefficient a
# Initialize gannet positions
gannet_positions = np.random.uniform(low=lower_bound, high=upper_bound, size=(num_gannets, num_variables))
best_solution = None
best_fitness = np.inf
def calculate_fitness(positions):
    # Calculate fitness values for each gannet's position
    fitness_values = np.sum(positions ** 2, axis=1)
    return fitness_values
for iteration in range(max_iterations):
    # Calculate fitness values for each gannet
    fitness_values = calculate_fitness(gannet_positions)  

    # Update the best solution
    min_fitness_index = np.argmin(fitness_values)
    if fitness_values[min_fitness_index] < best_fitness:
        best_fitness = fitness_values[min_fitness_index]
        best_solution = gannet_positions[min_fitness_index]

    # Calculate a for each gannet
    a_values = a_max - iteration * ((a_max - 1) / max_iterations)

    # Update gannet positions using their acceleration
    for i in range(num_gannets):
        for j in range(num_variables):
            r1 = np.random.random()
            r2 = np.random.random()
            r3 = np.random.random()
            C1 = 2 * r3
            D = np.abs(C1 * best_solution[j] - gannet_positions[i, j])
            gannet_positions[i, j] = (
                best_solution[j] - a_values * D * np.exp(-0.5 * (iteration / max_iterations))
                * np.cos(2 * np.pi * r2)
            )
            gannet_positions[i, j] = np.clip(gannet_positions[i, j], lower_bound, upper_bound)

y_preds = model.predict(X_test)
# Convert predicted probabilities to class labels
pred_labels = np.argmax(y_preds, axis=1)
# Convert true labels to class labels
true_labels = np.argmax(y_test, axis=1)
print("Best solution:", best_solution)
print("Best fitness:", best_fitness)


547/547 [==============================] - 1s 1ms/step
Best solution: [-5.97473312e-08  9.23114844e-09  6.21194902e-08  4.30941335e-08
  1.61444824e-08 -6.69460033e-09 -1.59353292e-08  7.86541929e-08
  4.73096226e-08 -6.49997958e-08]
Best fitness: 2.6174985351895094e-14


## *Result*

In [18]:
# Calculate accuracy
accuracy = accuracy_score(true_labels, pred_labels)
print("Accuracy:", accuracy )
print()
print("Error rate:" , 1-accuracy)
print()
# Calculate precision
precision = precision_score(true_labels, pred_labels)
print("Precision:", precision)
print()
# Calculate recall
recall = recall_score(true_labels, pred_labels)
print("Recall:", recall)
print()
conf_matrix=confusion_matrix(true_labels, pred_labels)
# True Negative (TN), False Positive (FP), False Negative (FN), True Positive (TP)
TN, FN, FP, TP = conf_matrix.ravel()
# Calculate specificity
specificity = TN / (TN + FP)
# Calculate F1-score
f1 = f1_score(true_labels, pred_labels)
# Specify training parameters
batch_size = 50
epochs = 15

# Calculate estimated computation time
start_time = time.time()
iterations_per_epoch = X_train.shape[0] // batch_size
total_iterations = iterations_per_epoch * epochs
time_per_pass = 0.006  # Assumption: 10 ms per forward-backward pass

estimated_computation_time = time_per_pass * total_iterations
end_time = time.time()

print(f"Estimated Computation Time: {estimated_computation_time:.2f} seconds")
print("F1-score:", f1)
print()
print("Specificity :",specificity)
print()
print("ROC :",roc_auc_score(true_labels, pred_labels))
print()
print("confusion_matrix")
print(confusion_matrix(true_labels, pred_labels))
print()
print("classification_report")
print(classification_report(true_labels, pred_labels))

Accuracy: 0.9965714285714286

Error rate: 0.0034285714285714475

Precision: 0.995823600198202

Recall: 0.9999289217428389

F1-score: 0.9978720385870337

Specificity : 0.9997035280166024

ROC : 0.9913663845088428

confusion_matrix
[[ 3372    59]
 [    1 14068]]

classification_report
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      3431
           1       1.00      1.00      1.00     14069

    accuracy                           1.00     17500
   macro avg       1.00      0.99      0.99     17500
weighted avg       1.00      1.00      1.00     17500

